In [142]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [143]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [144]:
sample = pd.read_csv('../input/2022-ml-midterm-p1/submit.csv')
test = pd.read_csv('../input/2022-ml-midterm-p1/test.csv')
train = pd.read_csv('../input/2022-ml-midterm-p1/train.csv')

In [145]:
sample.head()
# 심장 질환 '여부'를 판단하는 것이므로 1보다 크면 전부 '유'로 취급
train.target[train.target > 0] = 1
train.head()

In [146]:
test.head()

In [147]:
train.head()

In [148]:
train = train.drop(['index'], axis = 1)
test = test.drop(['index'], axis = 1)

In [149]:
trainX = train.drop(['target'], axis = 1)
trainY = train['target']

In [150]:
train.info()

In [151]:
test.info()

In [152]:
# 데이터 값 정규화
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
trainX_sc = scaler.fit_transform(trainX)
test_sc = scaler.transform(test)

In [153]:
# from sklearn.linear_model import LogisticRegression
# logis = LogisticRegression()
# logis.fit(trainX_sc, trainY)
# pred = logis.predict(test_sc)
# -> 0.80327

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lda = LinearDiscriminantAnalysis()
# trainX_sc_lda = lda.fit_transform(trainX_sc, trainY)
# test_sc_lda = lda.transform(test_sc)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX_sc, trainY, test_size=0.33, random_state=42)


from sklearn.ensemble import RandomForestClassifier
# 랜덤포레스트분류기 호출
clf = RandomForestClassifier(max_depth = 2, random_state=42)
# trainX_sc,trainY 로 학습시키기
clf.fit(trainX_sc, trainY)
print(clf.score(X_test, y_test))
# 모델로 test 예측하기
pred = clf.predict(test_sc)

# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=10, weights = 'distance')
# neigh.fit(trainX_sc, trainY)
# pred = neigh.predict(test_sc)

In [154]:
sample['target'] = pred


print(sample)
sample.to_csv('submission.csv', index = False)

In [155]:
sample.head()

logistic regression, LDA+knn으로 예측한 성능보다 random forest로 예측한 성능이 더 좋게 나왔다.
데이터의 크기가 작아 random하게 forest를 생성하는 것이 우수한 성능을 보이는 걸로 분석했다.

# Review
마지막 마크다운 보면 마음이 급한 게 여기까지 느껴지는 코드..
실은 내가 왜 random forest를 선택했는지도 모르겠고
이렇게 푼 게 왜 리더보드 1등을 차지했는지도 모르겠다..
1. 심장 질환 '여부'를 판단하는 것이므로 1보다 크면 전부 '유'로 취급
➡️ 시험 보기 직전에 작년 시험문제 엄청 조금 훑었는데 이 부분을 잘 기억해둬서 좋은 성능을 낼 수 있었던 것 같다. '여부'에 focus !
2. 데이터의 범위가 0.xx ~ 286 정도로 광범위하기 때문에 RobustScaler를 사용하여 정규화를 시켜주었다.
3. RandomForestClassifier(max_depth = 2, random_state = 42)가 0.90163의 성능을 보였다.
4. train_test_split 하지 않은 데이터로 학습시킨 모델의 성능이 더 우수했다.


⭐️ **A value is trying to be set on a copy of a slice from a DataFrame 
error 뜨는 거 대면 실습에서 조교님께 질문 드려야겠다**
